In [1]:
%load_ext autoreload
%load_ext line_profiler

In [2]:
from wordle_game import wordle, solver, helper

In [3]:
import os
import cProfile, pstats
from multiprocessing import Pool
from functools import partial

import numpy as np
from tqdm import tqdm

In [4]:
wordlist = wordle.WordList()

In [5]:
wordlist.answers[0], wordlist.acceptable_guesses[0]

('aback', 'aahed')

In [6]:
wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

array([2, 1, 0, 0, 0])

In [7]:
helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

5

## Profiling

In [8]:
%timeit wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

4.73 µs ± 151 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%timeit helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

599 ns ± 1.23 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%lprun -f wordle.generate_state wordle.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

In [11]:
def profiler():
    for i in range(100000):
        helper.generate_state(wordlist.answers[0], wordlist.acceptable_guesses[0])

In [12]:
cProfile.runctx("profiler()", globals(), locals(), "Profile.prof")

In [13]:
s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("cumtime").print_stats()

Mon Jan 31 21:11:30 2022    Profile.prof

         1400004 function calls in 0.304 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.304    0.304 {built-in method builtins.exec}
        1    0.000    0.000    0.304    0.304 <string>:1(<module>)
        1    0.024    0.024    0.304    0.304 858601308.py:1(profiler)
   100000    0.020    0.000    0.279    0.000 {built-in method wordle_game.helper.generate_state}
   100000    0.017    0.000    0.259    0.000 helper.pyx:35(generate_state (wrapper))
   100000    0.179    0.000    0.242    0.000 helper.pyx:35(generate_state)
  1100000    0.063    0.000    0.063    0.000 helper.pyx:32(map_char)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# Testing

In [14]:
game = wordle.WordleGame(target='perky')

In [15]:
solve = solver.MaxEntropy()

In [16]:
game.add_state("soare","BBBYY")
print(game)
solve.guess(game)

SOARE



'direr'

In [17]:
%lprun -f solve.guess solve.guess(game)

In [18]:
game.add_state("threw","BBGYB")
print(game)
solve.guess(game)

SOARE
THREW



'dampy'

In [19]:
game.add_state("nerdy","BGGBG")
print(game)
solve.guess(game)

SOARE
THREW
NERDY



'abamp'

In [20]:
game.add_state("mercy","BGGBG")
print(game)
solve.guess(game)

SOARE
THREW
NERDY
MERCY



'jerky'

In [21]:
solver.filter_answers(game)

['jerky', 'perky']

# Simulate

In [22]:
solutions = solve.simulate()
np.save('../data/maxentropy_solutions.npy', solutions)

100%|██████████| 2315/2315 [04:28<00:00,  8.61it/s]


In [23]:
solutions = np.load('../data/maxentropy_solutions.npy')

In [24]:
unique, counts = np.unique(solutions, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  -1   12]
 [   2   31]
 [   3 1085]
 [   4 1104]
 [   5   82]
 [   6    1]]


In [25]:
# Average number of guesses
np.sum(unique[1:]*counts[1:])/counts[1:].sum()

3.538428137212332

```python
cpu = os.cpu_count() - 1
cpu = cpu if cpu > 0 else cpu


results = []

for target in tqdm(wordlist.answers):
    with Pool(os.cpu_count() - 1) as pool:
        results.extend(pool.map(partial(wordle_game.helper.generate_state, target), wordlist.acceptable_guesses))
        pool.close()
        pool.join()
        
```